In [1]:
#James Chartouni
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline



In [41]:
data= pd.read_csv("cleaned_input/train_consolidated.csv")
test = pd.read_csv("cleaned_input/test_consolidated.csv")
test_msno = test.index;
train_msno = data.index;
test_msno_df = test['msno']
test = test.drop(["is_churn", 'msno', 'Unnamed: 0'], axis=1)
data = data.drop(["Unnamed: 0"], axis=1)

In [42]:
test.head()
test_msno_df.head()

0    4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=
1    aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=
2    rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=
3    WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=
4    aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=
Name: msno, dtype: object

In [32]:
data.head()

,msno,is_churn,trans_count,date_min,date_max,date_count,num_25_sum,num_50_sum,num_75_sum,num_985_sum,...,payment_method_id_encoded33,payment_method_id_encoded34,payment_method_id_encoded35,payment_method_id_encoded36,is_auto_renew_encoded0,is_auto_renew_encoded1,is_auto_renew_encoded2,is_cancel_encoded0,is_cancel_encoded1,is_cancel_encoded2
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,2,20161106.0,20170208.0,6.0,51.0,16.0,19.0,11.0,...,1,0,0,0,0,1,0,0,1,0
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,23,20150101.0,20170319.0,11.0,447.0,157.0,121.0,178.0,...,0,1,0,0,0,0,1,0,0,1
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,10,20160202.0,20170131.0,10.0,1570.0,950.0,302.0,479.0,...,0,1,0,0,0,0,1,0,0,1
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,3,20150101.0,20170331.0,11.0,7354.0,2443.0,1803.0,1655.0,...,1,0,0,0,0,1,0,0,1,0
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,9,20150101.0,20170331.0,11.0,1408.0,720.0,689.0,3249.0,...,1,0,0,0,0,1,0,0,1,0


In [5]:
data = data.drop(["Unnamed: 0"], axis=1)
test = test.drop(["Unnamed: 0"], axis=1)
data.shape

(504818, 125)

In [6]:
data.columns

Index(['msno', 'is_churn', 'trans_count', 'date_min', 'date_max', 'date_count',
       'num_25_sum', 'num_50_sum', 'num_75_sum', 'num_985_sum',
       ...
       'payment_method_id_encoded33', 'payment_method_id_encoded34',
       'payment_method_id_encoded35', 'payment_method_id_encoded36',
       'is_auto_renew_encoded0', 'is_auto_renew_encoded1',
       'is_auto_renew_encoded2', 'is_cancel_encoded0', 'is_cancel_encoded1',
       'is_cancel_encoded2'],
      dtype='object', length=125)

In [7]:
np.sum(np.sum(data.isnull())) # adding them row wise and then column wise. No value is none

115770

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(['is_churn', 'msno'], axis=1),data['is_churn'], train_size=0.8, test_size=0.2, random_state=345678) 

In [9]:
'''from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler();
scale.fit(X_train);
X_train = scale.transform(X_train);
X_test = scale.transform(X_test);'''

'from sklearn.preprocessing import MinMaxScaler\nscale = MinMaxScaler();\nscale.fit(X_train);\nX_train = scale.transform(X_train);\nX_test = scale.transform(X_test);'

In [10]:
import xgboost as xgb
from sklearn import metrics
from xgboost import XGBClassifier

In [11]:
#colsample_bytree=0.636221066412, gamma=0.859598446221, max_depth=7, min_child_weight=3, reg_alpha=3.59003832809, reg_lambda=0.312604869001, subsample=0.586018102037, score=-0.07374870982572349, total=
params = {'n_jobs':-1, 'max_depth':7, 'silent':1, 'objective':'binary:logistic', 'colsample_bytree':0.63622106641, 'gamma':0.859598446221, 'min_child_weight':3, "reg_alpha":3.59003832809, 'reg_lambda':0.312604869001, 'subsample':0.586018102037}
model = XGBClassifier()
print(model.set_params(**params))
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.63622106641, gamma=0.859598446221,
       learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=3, missing=None, n_estimators=100, n_jobs=-1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=3.59003832809, reg_lambda=0.312604869001,
       scale_pos_weight=1, seed=None, silent=0, subsample=0.586018102037)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.63622106641, gamma=0.859598446221,
       learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=3, missing=None, n_estimators=100, n_jobs=-1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=3.59003832809, reg_lambda=0.312604869001,
       scale_pos_weight=1, seed=None, silent=0, subsample=0.586018102037)

In [21]:
print(len(test_msno))
print(len(X_test))
test.head()

907471
100964


,msno,is_churn,trans_count,date_min,date_max,date_count,num_25_sum,num_50_sum,num_75_sum,num_985_sum,...,payment_method_id_encoded33,payment_method_id_encoded34,payment_method_id_encoded35,payment_method_id_encoded36,is_auto_renew_encoded0,is_auto_renew_encoded1,is_auto_renew_encoded2,is_cancel_encoded0,is_cancel_encoded1,is_cancel_encoded2
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0,21,20150719.0,20170115.0,2.0,8.0,2.0,1.0,0.0,...,0,0,0,1,0,0,1,0,1,0
1,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0,23,20150103.0,20170331.0,11.0,17694.0,3584.0,2536.0,2665.0,...,0,0,0,0,0,0,1,0,1,0
2,rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=,0,11,20150102.0,20170328.0,11.0,392.0,85.0,40.0,37.0,...,0,0,0,1,0,0,1,0,1,0
3,WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=,0,17,20150519.0,20170331.0,11.0,1040.0,282.0,182.0,188.0,...,0,0,0,1,0,0,1,0,1,0
4,aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=,0,4,20161226.0,20170216.0,4.0,12.0,7.0,4.0,3.0,...,0,0,0,0,0,0,1,0,1,0


In [33]:
y_pred = model.predict(test)

In [34]:
print(len(y_pred))

907471


In [64]:
submission = pd.DataFrame(y_pred).rename(columns={0:'is_churn'})
test_msno_df = pd.DataFrame(test_msno_df).rename(columns={0:'msno'})
result = pd.concat([test_msno_df,submission], axis=1)
result.head()

,msno,is_churn
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0
1,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0
2,rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=,0
3,WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=,0
4,aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=,0


In [66]:
result.to_csv("output/submission.csv")

In [67]:
result.shape

(907471, 2)